In [3]:
import torch

# Check if a GPU is available
is_available = torch.cuda.is_available()
print(f"GPU available: {is_available}")

if is_available:
    # Get the name of the GPU
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    # Get the number of GPUs
    print(f"Device count: {torch.cuda.device_count()}")

GPU available: False


# Reddit Business Ideas Finder

This notebook contains code to search Reddit for business ideas based on a given keyword. It will:
1. Search for relevant subreddits related to the keyword
2. Find top posts in those subreddits
3. Analyze discussions to identify potential business opportunities
4. Return structured business ideas with supporting data

In [ ]:
# Install required packages
# Uncomment the line below to install dependencies
# !pip install praw anthropic python-dotenv

import praw
import os
from typing import List, Dict, Any
from datetime import datetime
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()

In [ ]:
def search_relevant_subreddits(reddit: praw.Reddit, keyword: str, limit: int = 10) -> List[Dict[str, Any]]:
    """
    Search for subreddits related to the given keyword.
    
    Args:
        reddit: Authenticated Reddit client instance
        keyword: The keyword to search for
        limit: Maximum number of subreddits to return (default: 10)
    
    Returns:
        List of dictionaries containing subreddit information:
        - name: Subreddit name
        - subscribers: Number of subscribers
        - description: Subreddit description
        - url: URL to the subreddit
    """
    subreddits = []
    
    try:
        for subreddit in reddit.subreddits.search(keyword, limit=limit):
            subreddits.append({
                'name': subreddit.display_name,
                'subscribers': subreddit.subscribers,
                'description': subreddit.public_description,
                'url': f"https://reddit.com/r/{subreddit.display_name}"
            })
    except Exception as e:
        print(f"Error searching subreddits: {e}")
    
    return sorted(subreddits, key=lambda x: x['subscribers'], reverse=True)

In [ ]:
def get_top_comments(reddit: praw.Reddit, post_url: str, limit: int = 10) -> List[Dict[str, Any]]:
    """
    Get top comments from a Reddit post.
    
    Args:
        reddit: Authenticated Reddit client instance
        post_url: URL or permalink of the post
        limit: Maximum number of top comments to retrieve (default: 10)
    
    Returns:
        List of dictionaries containing comment information:
        - body: Comment text
        - score: Comment score
        - author: Comment author username
    """
    comments = []
    
    try:
        submission = reddit.submission(url=post_url)
        submission.comments.replace_more(limit=0)  # Remove "load more comments" objects
        
        for comment in submission.comments[:limit]:
            if hasattr(comment, 'body'):
                comments.append({
                    'body': comment.body[:300] if comment.body else '',  # Limit text length
                    'score': comment.score,
                    'author': str(comment.author) if comment.author else '[deleted]'
                })
    except Exception as e:
        print(f"Error fetching comments: {e}")
    
    return sorted(comments, key=lambda x: x['score'], reverse=True)

In [ ]:
def find_business_ideas_from_reddit(keyword: str, 
                                     num_subreddits: int = 5,
                                     posts_per_subreddit: int = 10,
                                     time_filter: str = 'month') -> Dict[str, Any]:
    """
    Main function to find business ideas from Reddit based on a keyword.
    
    This function orchestrates the entire process:
    1. Searches for relevant subreddits
    2. Finds top posts in those subreddits
    3. Analyzes discussions to extract business opportunities
    4. Returns structured business ideas with supporting data
    
    Args:
        keyword: The keyword/niche to search for
        num_subreddits: Number of subreddits to search (default: 5)
        posts_per_subreddit: Number of posts to fetch per subreddit (default: 10)
        time_filter: Time filter for posts ('month', 'week', 'year', etc.)
    
    Returns:
        Dictionary containing:
        - keyword: The search keyword
        - subreddits: List of relevant subreddits found
        - posts: All posts collected
        - business_ideas: List of identified business opportunities
        - metadata: Search metadata (timestamp, counts, etc.)
    """
    print(f"🔍 Searching Reddit for business ideas related to: '{keyword}'")
    
    # Initialize Reddit client
    reddit = initialize_reddit_client()
    
    # Step 1: Find relevant subreddits
    print(f"\n📍 Finding relevant subreddits...")
    subreddits = search_relevant_subreddits(reddit, keyword, limit=num_subreddits)
    print(f"Found {len(subreddits)} subreddits")
    
    # Step 2: Search posts in each subreddit
    print(f"\n📝 Searching posts in subreddits...")
    all_posts = []
    for subreddit in subreddits[:num_subreddits]:
        print(f"  - Searching r/{subreddit['name']}...")
        posts = search_posts_in_subreddit(
            reddit, 
            subreddit['name'], 
            keyword, 
            limit=posts_per_subreddit,
            time_filter=time_filter
        )
        all_posts.extend(posts)
    
    print(f"Found {len(all_posts)} total posts")
    
    # Step 3: Analyze for business ideas
    print(f"\n🤖 Analyzing posts with AI to extract business ideas...")
    business_ideas = analyze_for_business_ideas(all_posts, keyword)
    print(f"Identified {len(business_ideas)} business ideas")
    
    # Compile results
    results = {
        'keyword': keyword,
        'search_metadata': {
            'timestamp': datetime.now().isoformat(),
            'num_subreddits_searched': len(subreddits),
            'num_posts_analyzed': len(all_posts),
            'time_filter': time_filter
        },
        'subreddits': subreddits,
        'top_posts': sorted(all_posts, key=lambda x: x['score'], reverse=True)[:20],
        'business_ideas': business_ideas
    }
    
    return results

In [ ]:
def save_results_to_file(results: Dict[str, Any], filename: str = None) -> str:
    """
    Save the business ideas results to a JSON file.
    
    Args:
        results: Dictionary returned from find_business_ideas_from_reddit()
        filename: Optional custom filename. If not provided, generates one from keyword and timestamp
    
    Returns:
        Path to the saved file
    """
    if filename is None:
        keyword_clean = results['keyword'].replace(' ', '_').lower()
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"reddit_business_ideas_{keyword_clean}_{timestamp}.json"
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Results saved to: {filename}")
    return filename

In [ ]:
# Alternative: Quick test with any keyword
# Replace 'your_keyword_here' with any topic you want to explore

# keyword = input("Enter a keyword to search for business ideas: ")
# results = find_business_ideas_from_reddit(keyword)
# display_results(results)

## Output Structure

The `find_business_ideas_from_reddit()` function returns a dictionary with the following structure:

```json
{
  "keyword": "your search keyword",
  "search_metadata": {
    "timestamp": "2026-01-13T...",
    "num_subreddits_searched": 5,
    "num_posts_analyzed": 50,
    "time_filter": "month"
  },
  "subreddits": [
    {
      "name": "subreddit_name",
      "subscribers": 100000,
      "description": "Subreddit description",
      "url": "https://reddit.com/r/subreddit_name"
    }
  ],
  "top_posts": [
    {
      "title": "Post title",
      "score": 1500,
      "num_comments": 200,
      "url": "https://...",
      "permalink": "https://reddit.com/r/.../comments/...",
      "created_utc": "2026-01-13T...",
      "selftext": "Post content...",
      "author": "username"
    }
  ],
  "business_ideas": [
    {
      "idea_title": "Business Idea Title",
      "description": "Detailed description...",
      "target_audience": "Who this serves...",
      "pain_point": "Problem it solves...",
      "evidence": "Quotes from Reddit...",
      "potential": "High/Medium/Low",
      "keywords": ["tag1", "tag2"]
    }
  ]
}
```

### Tips for Best Results:

1. **Choose specific keywords**: Instead of broad terms like "business", use specific niches like "AI content creation" or "eco-friendly packaging"
2. **Adjust time filters**: Use 'week' for trending topics, 'year' or 'all' for established niches
3. **Increase post limits**: More posts = better analysis, but slower execution
4. **Review subreddits**: The most active subreddits often have the best discussions
5. **Cross-reference ideas**: Look for patterns across multiple posts and subreddits

In [ ]:
# Example: Search for business ideas related to "sustainable fashion"
# Uncomment and run the code below after setting up your API credentials

# keyword = "sustainable fashion"
# results = find_business_ideas_from_reddit(
#     keyword=keyword,
#     num_subreddits=5,
#     posts_per_subreddit=15,
#     time_filter='month'
# )
# 
# # Display the results in a formatted way
# display_results(results)
# 
# # Save results to JSON file
# save_results_to_file(results)

## Usage Example

To use this Reddit Business Ideas Finder, you need to set up the following environment variables:

### Required Environment Variables:

Create a `.env` file in the same directory with:

```
REDDIT_CLIENT_ID=your_reddit_client_id
REDDIT_CLIENT_SECRET=your_reddit_client_secret
REDDIT_USER_AGENT=YourAppName/1.0
ANTHROPIC_API_KEY=your_anthropic_api_key
```

### Getting Reddit API Credentials:

1. Go to https://www.reddit.com/prefs/apps
2. Click "Create App" or "Create Another App"
3. Select "script" as the app type
4. Fill in the name and description
5. Copy the client ID (under the app name) and client secret

### Getting Anthropic API Key:

1. Sign up at https://console.anthropic.com
2. Navigate to API Keys
3. Create a new API key

### Example Usage:

```python
# Search for business ideas related to a keyword
results = find_business_ideas_from_reddit(
    keyword="AI automation",
    num_subreddits=5,
    posts_per_subreddit=15,
    time_filter='month'
)

# Display the results
display_results(results)

# Save to file
save_results_to_file(results)
```

In [ ]:
def display_results(results: Dict[str, Any]) -> None:
    """
    Display the business ideas search results in a readable format.
    
    Args:
        results: Dictionary returned from find_business_ideas_from_reddit()
    """
    print("=" * 80)
    print(f"REDDIT BUSINESS IDEAS ANALYSIS: {results['keyword'].upper()}")
    print("=" * 80)
    
    # Display metadata
    metadata = results['search_metadata']
    print(f"\n📊 Search Summary:")
    print(f"   Timestamp: {metadata['timestamp']}")
    print(f"   Subreddits searched: {metadata['num_subreddits_searched']}")
    print(f"   Posts analyzed: {metadata['num_posts_analyzed']}")
    print(f"   Time filter: {metadata['time_filter']}")
    
    # Display subreddits
    print(f"\n📍 Relevant Subreddits:")
    for i, sub in enumerate(results['subreddits'][:5], 1):
        print(f"   {i}. r/{sub['name']} ({sub['subscribers']:,} subscribers)")
        print(f"      {sub['description'][:100]}...")
    
    # Display business ideas
    print(f"\n💡 Business Ideas ({len(results['business_ideas'])} found):")
    print("=" * 80)
    
    for i, idea in enumerate(results['business_ideas'], 1):
        print(f"\n{i}. {idea.get('idea_title', idea.get('title', 'Untitled Idea'))}")
        print(f"   {'-' * 70}")
        print(f"   Description: {idea.get('description', 'N/A')}")
        print(f"   Target Audience: {idea.get('target_audience', 'N/A')}")
        print(f"   Pain Point: {idea.get('pain_point', 'N/A')}")
        print(f"   Market Potential: {idea.get('potential', idea.get('market_potential', 'N/A'))}")
        if 'evidence' in idea:
            print(f"   Evidence: {idea['evidence'][:150]}...")
        if 'keywords' in idea:
            keywords = idea['keywords'] if isinstance(idea['keywords'], str) else ', '.join(idea['keywords'])
            print(f"   Keywords: {keywords}")
    
    print("\n" + "=" * 80)

In [ ]:
def analyze_for_business_ideas(posts_data: List[Dict[str, Any]], keyword: str) -> List[Dict[str, Any]]:
    """
    Analyze Reddit posts and comments to extract potential business ideas.
    
    This function uses Claude AI to analyze discussions and identify business opportunities,
    pain points, feature requests, and market gaps related to the keyword.
    
    Args:
        posts_data: List of post dictionaries with titles, content, and comments
        keyword: The keyword/niche being analyzed
    
    Returns:
        List of dictionaries containing business ideas:
        - idea_title: Brief title of the business idea
        - description: Detailed description of the opportunity
        - target_audience: Who would benefit from this
        - pain_point: What problem it solves
        - evidence: Supporting quotes/data from Reddit
        - potential: Estimated market potential (High/Medium/Low)
        - keywords: Related keywords and tags
    """
    from anthropic import Anthropic
    
    # Initialize Anthropic client
    client = Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))
    
    # Prepare context from Reddit posts
    context = f"Keyword: {keyword}\n\n"
    for i, post in enumerate(posts_data[:10], 1):  # Limit to top 10 posts
        context += f"Post {i}:\n"
        context += f"Title: {post['title']}\n"
        context += f"Score: {post['score']} | Comments: {post['num_comments']}\n"
        if post['selftext']:
            context += f"Content: {post['selftext'][:300]}...\n"
        context += "\n"
    
    # Create prompt for Claude
    prompt = f"""Analyze the following Reddit discussions about '{keyword}' and identify 3-5 concrete business ideas or opportunities.

{context}

For each business idea, provide:
1. A catchy title
2. Detailed description of the opportunity
3. Target audience
4. Specific pain point it addresses
5. Evidence from the Reddit discussions
6. Market potential (High/Medium/Low)
7. Related keywords

Return the response as a JSON array of business ideas."""
    
    try:
        response = client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=4000,
            messages=[{
                "role": "user",
                "content": prompt
            }]
        )
        
        # Parse the response
        response_text = response.content[0].text
        
        # Try to extract JSON from the response
        import re
        json_match = re.search(r'\[.*\]', response_text, re.DOTALL)
        if json_match:
            business_ideas = json.loads(json_match.group())
            return business_ideas
        else:
            print("Could not parse JSON response from Claude")
            return []
            
    except Exception as e:
        print(f"Error analyzing business ideas with AI: {e}")
        return []

In [ ]:
def search_posts_in_subreddit(reddit: praw.Reddit, subreddit_name: str, keyword: str, 
                              limit: int = 25, time_filter: str = 'month') -> List[Dict[str, Any]]:
    """
    Search for posts containing the keyword in a specific subreddit.
    
    Args:
        reddit: Authenticated Reddit client instance
        subreddit_name: Name of the subreddit to search
        keyword: The keyword to search for in posts
        limit: Maximum number of posts to return (default: 25)
        time_filter: Time filter for top posts ('hour', 'day', 'week', 'month', 'year', 'all')
    
    Returns:
        List of dictionaries containing post information:
        - title: Post title
        - score: Post score (upvotes - downvotes)
        - num_comments: Number of comments
        - url: URL to the post
        - created_utc: Post creation timestamp
        - selftext: Post content/body
        - author: Post author username
    """
    posts = []
    
    try:
        subreddit = reddit.subreddit(subreddit_name)
        
        # Search posts containing the keyword
        for post in subreddit.search(keyword, sort='top', time_filter=time_filter, limit=limit):
            posts.append({
                'title': post.title,
                'score': post.score,
                'num_comments': post.num_comments,
                'url': post.url,
                'permalink': f"https://reddit.com{post.permalink}",
                'created_utc': datetime.fromtimestamp(post.created_utc).isoformat(),
                'selftext': post.selftext[:500] if post.selftext else '',  # Limit text length
                'author': str(post.author) if post.author else '[deleted]'
            })
    except Exception as e:
        print(f"Error searching posts in r/{subreddit_name}: {e}")
    
    return sorted(posts, key=lambda x: x['score'], reverse=True)

In [ ]:
def initialize_reddit_client():
    """
    Initialize Reddit API client using PRAW (Python Reddit API Wrapper).
    
    Requires environment variables:
    - REDDIT_CLIENT_ID: Your Reddit app client ID
    - REDDIT_CLIENT_SECRET: Your Reddit app client secret
    - REDDIT_USER_AGENT: A unique identifier for your app
    
    Returns:
        praw.Reddit: Authenticated Reddit client instance
    """
    return praw.Reddit(
        client_id=os.getenv('REDDIT_CLIENT_ID'),
        client_secret=os.getenv('REDDIT_CLIENT_SECRET'),
        user_agent=os.getenv('REDDIT_USER_AGENT', 'BusinessIdeaFinder/1.0')
    )